In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import numpy
import math

In [13]:
# Defining Parameters
TARGET_SIZE = (256, 256)
COLOR_MODE = 'grayscale'
BATCH_SIZE = 5

In [14]:
# Load Images
data_gen = ImageDataGenerator(rescale=1.0/255)
train_it = data_gen.flow_from_directory("Covid19-dataset/train", target_size=TARGET_SIZE, color_mode=COLOR_MODE, class_mode='sparse', batch_size=BATCH_SIZE)
validate_it = data_gen.flow_from_directory("Covid19-dataset/test", target_size=TARGET_SIZE, color_mode=COLOR_MODE, class_mode='sparse', batch_size=BATCH_SIZE)


print(train_it.image_shape)
print(train_it.num_classes)

Found 253 images belonging to 3 classes.
Found 66 images belonging to 3 classes.
(256, 256, 1)
3


In [15]:
# Create Model
model = Sequential()
model.add(layers.Input(shape=(256, 256, 1)))
model.add(layers.Conv2D(3, 5, strides=3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(layers.Conv2D(3, 5, strides=3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
    
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 84, 84, 3)      │            78 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 41, 41, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 3)      │           228 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 108)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           327 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 633 (2.47 KB)

 Trainable params: 633 (2.47 KB)

 Non-trainable params: 0 (0.00 B)

None


In [16]:
# Fit and Evaluate Model
model.fit(
    train_it, steps_per_epoch=train_it.samples//BATCH_SIZE, 
    epochs=10, validation_data=validate_it, validation_steps=validate_it.samples//BATCH_SIZE
)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3041 - loss: 1.1510 - val_accuracy: 0.5692 - val_loss: 1.0813
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 206us/step - accuracy: 0.0000e+00 - loss: 1.1497 - val_accuracy: 0.0000e+00 - val_loss: 1.1038
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5455 - loss: 1.0802 - val_accuracy: 0.6000 - val_loss: 1.0526
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 172us/step - accuracy: 0.4000 - loss: 1.0573 - val_accuracy: 0.0000e+00 - val_loss: 1.2271
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6598 - loss: 1.0250 - val_accuracy: 0.6308 - val_loss: 0.9525
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 161us/step - accuracy: 0.6000 - loss: 0.9700 - val_accuracy: 1.0000 - val_loss: 0.7348
Epoch 7/10
24/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8595 - loss: 0.8706

2024-10-29 19:11:05.543438: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8241 - loss: 0.8388 - val_accuracy: 0.6769 - val_loss: 0.7131
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 163us/step - accuracy: 0.8000 - loss: 0.7493 - val_accuracy: 0.0000e+00 - val_loss: 1.4124
Epoch 9/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8763 - loss: 0.5439 - val_accuracy: 0.7385 - val_loss: 0.5701
Epoch 10/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 155us/step - accuracy: 0.6000 - loss: 0.5986 - val_accuracy: 1.0000 - val_loss: 0.6762
